# prologue

### set up notebook and load package

In [1]:
# for notebook plotting
%matplotlib inline 

# load what we need
import time
import timeit
import numpy as np
import CHIRPS.structures as strcts
import CHIRPS.routines as rt
import CHIRPS.reproducible as rp
import CHIRPS.boosting_scratch as bs
from IPython.display import Image
from sklearn.externals.six import StringIO
# from pydotplus import graph_from_dot_data

# demo datasets that ship with package. all from UCI unless stated otherwise
# import CHIRPS.datasets as ds
# ds.adult_data, ds.adult_samp_data, ds.adult_small_samp_data Large dataset ships with manageable sub samples
# ds.bankmark_data, ds.bankmark_samp_data
# ds.car_data
# ds.cardio_data this is the cardiotocography dataset
# ds.credit_data
# ds.german_data
# ds.lending_data, ds.lending_samp_data, ds.lending_small_samp_data, ds.lending_tiny_samp_data from Kaggle. see datasets_from_source file for links
# ds.nursery_data, ds.nursery_samp_data
# ds.rcdv_data, ds.rcdv_samp_data from US government see datasets_from_source file for links

### common config - can be ommitted if defaults are OK

In [2]:
# project_dir = 'V:\\whiteboxing\\examples' # defaults to a directory "whiteboxing" in the working directory
project_dir = 'C:\\Users\\Crutt\\Documents\\whiteboxing\\examples'
random_state_splits = 123 # one off for splitting the data into test / train
random_state = 123 # for everything else - e.g. building a new rf with same data

# Build a Random Forest Model to Predict and Explain
First, a wrapper is created for the dataset. Use one that ships with the package, or create your own.
Then split the data into training and (hold out) test set using the convenience functions in the package. These return an object that contain the split data in various representations, such as Pandas DataFrames and encoded, sparse matrices.

In [3]:
# load one of the included datasets
# project_dir will default to directory name CHIRPS in the working directory if not given
# random_state will default to 123
mydata = rp.datasets[0](random_state=random_state, project_dir=project_dir)
meta_data = mydata.get_meta()
save_path = meta_data['get_save_path']()

# split the data. here using a basic sampling method.
# the returned object is a wrapper class that contains
# the train and test splits for X and y

# also the the encoded versions of X_train and X_test that the rf will use
# this is because we prefer onehot encoded over allowing categorical vars to be represented as integer
# scikit would treat these as ordinal, which is inappropriate

# also some meta-data: priors for y, the indexes from the input data

# also some convenience functions for leave-one-out testing

# train test split - one off hard-coded random state.
# random state can be ommitted 
# and will default to the state held in the dataset container
# which defaults to 123 if ommitted in the constructor
train_index, test_index = mydata.get_tt_split_idx(random_state=random_state_splits)
# optionally, indexes can be ommitted and will default to scikit's train_test_split method
tt = mydata.tt_split(train_index, test_index)

In [152]:
# build a random forest, tuned for high accuracy
# rf = rp.forest_prep(ds_container=tt,
#                     meta_data=meta_data,
#                     save_path=save_path,
#                     plot_cm=True, plot_cm_norm=True)

# will need to create a XGBoost prep that finds the number of estimators and the learning rate

ds_container = tt

X_train=ds_container.X_train_enc
X_test=ds_container.X_test_enc
y_train=ds_container.y_train
y_test=ds_container.y_test

ne = 100

# xgb = bs.XGBClassifier(seed=random_state,
#                         max_depth=4,
#                         learning_rate=0.05,
#                         n_estimators=ne)

gb = bs.GradientBoostingClassifier()
gb.set_params(random_state=random_state,
            max_depth=1,
            learning_rate=1,
            n_estimators=ne)
gb.fit(X_train, y_train)

ad = bs.AdaBoostClassifier()
ad.set_params(random_state=random_state,
            learning_rate=1,
            n_estimators=ne,
            base_estimator=bs.DecisionTreeClassifier(max_depth=4))
ad.fit(X_train, y_train)

# print(bs.calculate_weights(ad.estimator_errors_))
# print(ad.estimator_errors_)
# for g in ad.staged_predict(X_test):
#     print(g)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          learning_rate=1, n_estimators=100, random_state=123)

In [153]:
# dot_data = StringIO()
# bs.export_graphviz(ad.estimators_[99],
#                 feature_names=meta_data['features_enc'],
#                 filled=True,
#                 rounded=True,
#                 out_file=dot_data)
# graph = graph_from_dot_data(dot_data.getvalue())


# Image(graph.create_png())

In [154]:
import math
import numpy as np

def somefunc(arr):
    def expit(err):
        err_val = math.exp(err) / (1 + math.exp(err))
        return(err_val)

    vexpit = np.vectorize(expit)
    return(vexpit(arr))



somefunc(gb.decision_function(X_test))

j = 2
print(math.exp(gb.init_.predict(X_test)[j][0]))
for g, h, k in zip(gb.staged_decision_function(X_test),
                   gb.staged_predict_proba(X_test),
                   gb.staged_predict(X_test)):
    print(g[j][0], math.exp(g[j][0]), h[j][1], k[j])

# https://stats.stackexchange.com/questions/157870/scikit-binomial-deviance-loss-function

0.31765612952968386
-2.033704089880192 0.1308499424424723 0.11570937710785514 0
-2.4389947452426655 0.08724851437696572 0.08024707619578804 0
-2.9390816087834946 0.05291430246070371 0.05025508945698697 0
-3.696650386809696 0.02480647957355913 0.024206013591835957 0
-3.1866311707064767 0.04131080583653489 0.03967192658041029 0
-3.2680104418097438 0.03808211834456893 0.03668507305115566 0
-3.3066706545503637 0.03663795131796786 0.03534305421809693 0
-3.1656819175462556 0.04218536506290421 0.04047779452396934 0
-3.1070842699166765 0.04473118945900883 0.04281597975664141 0
-3.0190292909265537 0.048848613147024544 0.04657355936282969 0
-2.994151843737312 0.050079083967703185 0.04769077370675773 0
-3.048781737425778 0.04741665515475624 0.04527009850511725 0
-3.0511150712534314 0.04730614524768048 0.04516935708086857 0
-3.0881904870791335 0.04558436531836127 0.04359702270842742 0
-3.2289987046966258 0.03959712736279055 0.038088915716070704 0
-3.1257901616913983 0.043902230054141796 0.04205588

In [102]:
# how is GB working to calculate the predict function
import numpy as np
n = 300
gbpreds = [gb.predict(X_test[i])[0] for i in range(n)]
reg_tots = np.empty(n)
for i in range(n):
    reg_tots[i] = sum([gb.estimators_[j][0].predict(X_test[i])[0] for j in range(ne)])

mn = np.inf
mx = -np.inf
for pred, tot in zip(gbpreds, reg_tots):
    if pred == 0:
        if mx < tot:
            mx = tot
    if pred == 1:
        if mn > tot:
            mn = tot
            
print(mx, mn)

11.206108458144511 11.804121017970653


In [6]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import tree
from sklearn.datasets import load_wine
from IPython.display import SVG
from graphviz import Source
from IPython.display import display
# load dataset
data = load_wine()

# feature matrix
X = data.data

# target vector
y = data.target

# class labels
labels = data.feature_names

# print dataset description
print(data.DESCR)
estimator = DecisionTreeClassifier()
estimator.fit(X, y)

graph = Source(tree.export_graphviz(estimator, out_file=None
   , feature_names=labels, class_names=['0', '1', '2'] 
   , filled = True))
display(SVG(graph.pipe(format='svg')))

ModuleNotFoundError: No module named 'graphviz'

In [6]:
# ad.estimators_[0].predict_proba(X_test)
ad.estimator_errors_
# gb.loss_(y_test, gb.predict(X_test))
bs.get_graph(ad.estimators_[0], n_classes=2)

NameError: name 'StringIO' is not defined

# Preparing unseen data

Again note:
test set has never been "seen" by random forest during training
test set has been only used to assess model (random forest) accuracy - no additional tuning after this
test set has not be involved in generating the explainer

## optional: memory and computation cost management
#### CHIRPS is time economical but memory intensive to compute for lots of instances at once
option 1: choose a smaller number of instances to explain

In [5]:
# control for async processes - each tree walk can be done in its own core
# and so can each explanation (e.g. rule conditions merge by hill-climbing)
# these will default to false if not passed explicitly to the explainer function
# on a multi-core machine there should be a good speed up for large batches
# when the batch_size advantage exceeds the overhead of setting up multi-processing
# timings will be printed to screen so you can see if it helps
forest_walk_async=False
chirps_explanation_async=False

# the number of instances can be controlled by
# batch_size - how many instances to explain at one time
batch_size = 1
# how many instances to explain in total from a test/unseen set
n_instances = 1

# this will normalise the above parameters to the size of the dataset
n_instances, n_batches = rt.batch_instance_ceiling(ds_container=tt, n_instances=n_instances, batch_size=batch_size)

# this gets the next batch out of the data_split_container according to the required number of instances
# all formats can be extracted, depending on the requirement
# unencoded, encoded (sparse matrix is the type returned by scikit), ordinary dense matrix also available
instances, _, instances_enc, instances_enc_matrix, labels = tt.get_next(batch_size, which_split='test') # default

option 2: just run the whole test set

In [6]:
# instances = tt.X_test; instances_enc = tt.X_test_enc; instances_enc_matrix = tt.X_test_enc_matrix; labels = tt.y_test

## Make predictions from the decision forest on the unseen data
Important point, no compromise on model accuracy

In [7]:
# get all the model predictions for the test instance(s) we're looking at
preds_idx = labels.index
preds = rf.predict(X=instances_enc)

# CHIRPS Step 1:
## Extract Tree Prediction Paths
### Fit a forest_walker object to the dataset and decision forest
This is a wrapper will extracts the paths of all the given instances. For CHIRPS, we want a large sample. The whole training set or other representative sample will do.

It can also report interesting statistics (treating the forest as a set of random tree-structured variables).

In [8]:
# wrapper object needs the decision forest itself and the dataset meta data (we have a convenience function for this)
f_walker = strcts.forest_walker(forest = rf, meta_data=meta_data)

Now the work of extracting all the paths for each instance is done

In [9]:
print('Walking forest for ' + str(len(labels)) + ' instances... (please wait)')

# set the timer
forest_walk_start_time = timeit.default_timer()

# do the walk - returns a batch_paths_container (even for just one instance)
# requires the X instances in a matrix (dense, ordinary numpy matrix) - this is available in the data_split_container
bp_container = f_walker.forest_walk(instances = instances_enc_matrix
                        , labels = preds # we're explaining the prediction, not the true label!
                        , forest_walk_async = forest_walk_async)

# stop the timer
forest_walk_end_time = timeit.default_timer()
forest_walk_elapsed_time = forest_walk_end_time - forest_walk_start_time

print('Forest Walk with async = ' + str(forest_walk_async))
print('Forest Walk time elapsed:', "{:0.4f}".format(forest_walk_elapsed_time), 'seconds')

Walking forest for 1 instances... (please wait)
Forest Walk with async = True
Forest Walk time elapsed: 2.0666 seconds


# CHIRPS Steps 2-4: 
## Freqent pattern mining of paths.
## Score and sort mined path segments.
## Merge path segments into one rule.

This is a wrapper object that will execute steps 2-4 on all the instance-paths in the batch_paths_container.

Note that true_divide warnings are OK. It just means that a continuous variable is unbounded in some way i.e. no greater/less than discontinuity is used in the CHIRPS explanation.

Note also, here we are using the training set to create the explainers. We could use a different dataset as long as it is representative of the training set that built the decision forest. Most important that we don't use the dataset that we wish to explain.

In [10]:
# get what the model predicts on the training sample
sample_labels = rf.predict(tt.X_train_enc)

# build CHIRPS and a rule for each instance represented in the batch paths container
CHIRPS = strcts.batch_CHIRPS_explainer(bp_container,
                                forest=rf,
                                sample_instances=tt.X_train_enc, # any representative sample can be used
                                # sample_labels=tt.y_train,  # any representative sample can be used
                                sample_labels=sample_labels,
                                meta_data=meta_data)

print('Running CHIRPS on a batch of ' + str(len(labels)) + ' instances... (please wait)')
# start a timer
ce_start_time = timeit.default_timer()

CHIRPS.batch_run_CHIRPS(chirps_explanation_async=chirps_explanation_async,
                        alpha_paths=0.9,
                        support_paths=0.1,
                        score_func=5,
                        disc_path_bins=4,
                        target_classes=preds,
                        merging_bootstraps=20,
                        pruning_bootstraps=20,
                        delta=0.1,
                        weighting='chisq')

ce_end_time = timeit.default_timer()
ce_elapsed_time = ce_end_time - ce_start_time
print('CHIRPS time elapsed:', "{:0.4f}".format(ce_elapsed_time), 'seconds')
print('CHIRPS with async = ' + str(chirps_explanation_async))

Running CHIRPS on a batch of 1 instances... (please wait)
CHIRPS time elapsed: 2.9593 seconds
CHIRPS with async = True


# Viewing and Evaluating CHIRPS explanations
Evaluation is done using unseen data to see how well the explanations generalise. The data_split_container object (tt) has a  leave-one-out function that is used during the routine to ensure that the datum we are explaining is excluded from the evaluation.

In [11]:
# iterate over all the test instances to determine the various scores using leave-one-out testing
print('evaluating found explanations')
print()
results_start_time = timeit.default_timer()

rt.evaluate_CHIRPS_explainers(CHIRPS, tt, tt.y_test.index,
                              forest=rf,
                              meta_data=meta_data,
                              eval_start_time=results_start_time,
                              print_to_screen=True, # set True when running single instances
                              eval_alt_labelings=True,
                              eval_rule_complements=True,
                              save_results_path=save_path,
                              dataset_name='test',
                              save_results_file='CHIRPS' + '_rnst_' + str(random_state),
                              save_CHIRPS=False)

results_end_time = timeit.default_timer()
results_elapsed_time = results_end_time - results_start_time
print('CHIRPS batch results eval time elapsed:', "{:0.4f}".format(results_elapsed_time), 'seconds')
# this completes the CHIRPS runs

evaluating found explanations

INSTANCE RESULTS
instance id: 33 with true class label: 1 (priority)

Model Results for Instance
target (predicted) class: 1 (priority)
target class prior (training data): 0.33076074972436603
forest vote share (unseen instance): 0.8753333333333333
forest vote margin (unseen instance): 0.7506666666666666
rule: has_nurs_critical False AND has_nurs_very_crit False AND has_nurs_improper False AND health_priority True AND parents_great_pret False
rule cardinality: 5

Estimated Results - Rule Training Sample. Algorith: greedy_stab
rule coverage (training data): 0.09371554575523705
rule xcoverage (training data): 0.09366391184573003
rule precision (training data): 1.0
rule stability (training data): 0.9941520467836257
rule recall (training data): 0.2833333333333333
rule f1 score (training data): 0.4415584415584416
rule lift (training data): 32.260745098039216
prior (training data): [0.33958104 0.33076075 0.3076075  0.02205072]
prior counts (training data): [616 

In [12]:
from pandas import Series
forest = rf
train_pred_labels = Series(forest.predict(tt.X_train_enc), index = tt.y_train.index)
CHIRPS.CHIRPS_explainers[0].evaluate_rule(rule='pruned', sample_instances=tt.X_train_enc, sample_labels=train_pred_labels)

[('has_nurs_critical', True, 0.5),
 ('has_nurs_very_crit', True, 0.5),
 ('has_nurs_improper', True, 0.5),
 ('health_not_recom', True, 0.5),
 ('health_recommended', True, 0.5),
 ('parents_great_pret', True, 0.5)]

In [13]:
CHIRPS.CHIRPS_explainers[0].posterior

array([[0.33958104, 0.33076075, 0.3076075 , 0.02205072],
       [0.        , 0.69463549, 0.25034388, 0.05502063],
       [0.        , 0.84309623, 0.07322176, 0.08368201],
       [0.        , 0.89634146, 0.        , 0.10365854],
       [0.        , 1.        , 0.        , 0.        ]])